In [1]:
from utils import *
import matplotlib.pyplot as plt

# Step 1: Take any pair of images of an object
obj = 'Arc_de_Triomphe_du_Carrousel_by_angle'
path_to_pair_metadata_train = 'data/pairs_metadata/doppelgangers/pairs_metadata/train_pairs_noflip.npy'

# pair = get_pairs_train(path_to_pair_metadata_train, obj)[0]

# # Step 2: Get the keypoints location of the pair
# parent_path_keypoints_location = 'data/matches_train_noflip/doppelgangers/loftr_matches/train_set_noflip/'
# keypoints_location = get_keypoints_location_pair(parent_path_keypoints_location, pair[1])

# # Step 3: take each image of the pair
# root_path_images = 'data/train_set_noflip/doppelgangers/images/train_set_noflip/'
# image1, image2 = get_images(pair[0][:2], root_path_images)

# # Step 4: resize and padding to 1024x1024 and crop according to keypoints
# image1_process = resize_and_pad_bottom_right(image1)
# image2_process = resize_and_pad_bottom_right(image2)

# crop1, crop2 = crop_matched_regions(image1_process, image2_process, **keypoints_location, threshold=0.6)

# image1_with_keypoints = write_keypoints_to_image(image1_process, keypoints_location['kpt0'])
# image2_with_keypoints = write_keypoints_to_image(image2_process, keypoints_location['kpt1'])

# # List of images and titles
# images = [
#     (image1, "Image 1"),
#     (image2, "Image 2"),
#     (crop1, "Cropped Region in Image 1"),
#     (crop2, "Cropped Region in Image 2"),
#     (image1_with_keypoints, "Image 1 with keypoints"),
#     (image2_with_keypoints, "Image 2 with keypoints"),
# ]

# # show images
# plt.figure(figsize=(10, 15))
# for i, (img, title) in enumerate(images, 1):
#     plt.subplot(3, 2, i)
#     plt.imshow(img)
#     plt.axis('off')
#     plt.title(title)

# plt.tight_layout()
# plt.show()


In [ ]:
pairs = get_pairs_train(path_to_pair_metadata_train, obj)

import os
from PIL import Image
from tqdm import tqdm
new_pairs = []

for pair in tqdm(pairs):
    # Step 2: Get the keypoints location of the pair
    try:
        # print(pair[1])
        # dùng để kết hợp với các file như 0.npy để lấy ra thông tin
        parent_path_keypoints_location = 'data/matches_train_noflip/doppelgangers/loftr_matches/train_set_noflip/'
        keypoints_location = get_keypoints_location_pair(parent_path_keypoints_location, pair[1])

        # Step 3: take each image of the pair
        # đường dẫn đến thư mục chứa từng vật thể (như khải hoàn môn, ...)
        root_path_images = 'data/train_set_noflip/doppelgangers/images/train_set_noflip/'
        image1, image2 = get_images(pair[0][:2], root_path_images)

        # Step 4: resize and padding to 1024x1024 and crop according to keypoints
        image1_process = resize_and_pad_bottom_right(image1)
        image2_process = resize_and_pad_bottom_right(image2)

        crop1, crop2 = crop_matched_regions(image1_process, image2_process, **keypoints_location, threshold=0.6)

        crop1_resize = cv2.resize(crop1, (224, 224))
        crop2_resize = cv2.resize(crop2, (224, 224))

        root1 = 'cropped_train/' + '/'.join(pair[0][0].split('/')[:2]) + '/'
        root2 = 'cropped_train/' + '/'.join(pair[0][1].split('/')[:2]) + '/'

        name_img1 = pair[0][0].split('/')[-1]
        name_img2 = pair[0][1].split('/')[-1]

        new_pair = []

        for root, name_img, img_array in zip([root1, root2], [name_img1, name_img2], [crop1, crop2]):
            if not os.path.exists(root):
                os.makedirs(root)
            
            img = Image.fromarray(img_array)
            img = img.convert('RGB')
            img.save(root + name_img)

            new_pair.append(root + name_img)

        new_pair.append(pair[0][2])

        new_pairs.append(new_pair)

    except:
        continue
        
new_pairs = np.array(new_pairs)
np.save('new_metadata/train_metadata.npy', new_pairs)
    

100%|██████████| 2426/2426 [02:10<00:00, 18.57it/s]


In [12]:
# Lặp qua từng cặp trong pairs
from tqdm import tqdm
for pair in tqdm(pairs):
    for i, path in enumerate(pair[:2]):  # Duyệt qua hai phần tử đầu tiên trong mỗi cặp
        image = Image.open(path)

        cleaned_path = path.replace("'", "")  # Loại bỏ dấu '
        cleaned_path = os.path.join('without_quote', cleaned_path)
        # Đổi tên file
        # os.rename(path, cleaned_path)

        # Lấy đường dẫn thư mục từ cleaned_path
        dir_name = os.path.dirname(cleaned_path)

        # Kiểm tra nếu thư mục không tồn tại thì tạo thư mục
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)

        image.save(cleaned_path)
        
        # Cập nhật lại path trong pair sau khi đổi tên
        pair[i] = cleaned_path

# In ra các cặp đã được thay đổi
print(pairs)

100%|██████████| 400/400 [00:03<00:00, 114.74it/s]

[['without_quote\\cropped_test/Arc_de_Triomphe_de_l%27%C3%89toile_by_angle/back_left/Back left view of the Arc de Triomphe, Paris 21 August 2013.jpg'
  'without_quote\\cropped_test/Arc_de_Triomphe_de_l%27%C3%89toile_by_angle/front_right/Arc de Triomphe, Paris 5-2013.jpeg'
  '0']
 ['without_quote\\cropped_test/Arc_de_Triomphe_de_l%27%C3%89toile_by_angle/back_right/Arc de Triomphe de lÉtoile, Paris October 2014.jpg'
  'without_quote\\cropped_test/Arc_de_Triomphe_de_l%27%C3%89toile_by_angle/front_left/Arc de Triomphe in 2013.03.jpg'
  '0']
 ['without_quote\\cropped_test/Arc_de_Triomphe_de_l%27%C3%89toile_by_angle/back/Arc de Triomphe 4, Paris December 2012.jpg'
  'without_quote\\cropped_test/Arc_de_Triomphe_de_l%27%C3%89toile_by_angle/front/Arc de Triomphe de lÉtoile - scan01.jpg'
  '0']
 ...
 ['without_quote\\cropped_test/Arc_de_Triomphe_de_l%27%C3%89toile_by_angle/front_left/ArcDeTriomphe.jpg'
  'without_quote\\cropped_test/Arc_de_Triomphe_de_l%27%C3%89toile_by_angle/front_left/Arc de T

In [13]:
np.save('without_quote/new_metadata/test_metadata.npy', pairs)